In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm
import random
import os
import math

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

SEED = 42
seed_everything(SEED)
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {DEVICE}")

# 하이퍼파라미터 [V4 Advanced - 학습 전략 개선]
BATCH_SIZE = 64
LR_BASE = 1e-3
WARMUP_EPOCHS = 3      # Warmup 에포크
EPOCHS_BASE = 40       # 30 -> 40 (충분한 학습)
D_MODEL = 128          # 원래대로
NHEAD = 4              # 원래대로
NUM_LAYERS = 2         # 원래대로
DROPOUT = 0.2          # 원래대로
MAX_SEQ_LEN = 30       # 원래대로
GRAD_CLIP = 1.0        # Gradient Clipping

In [9]:
# ======================================================
# 1. 경로 설정 및 Train 로드
# ======================================================
BASE_DIR = "./open_track1" # 데이터가 있는 폴더명 (환경에 맞게 수정)
if not os.path.exists(BASE_DIR): BASE_DIR = "." # 폴더 없으면 현재 경로

TRAIN_PATH = os.path.join(BASE_DIR, "train.csv")
TEST_META_PATH = os.path.join(BASE_DIR, "test.csv") # 사용자님이 올리신 파일명
MATCH_PATH = os.path.join(BASE_DIR, "match_info.csv")

# 1. Train 로드
train_df = pd.read_csv(TRAIN_PATH)
print(f"✅ Train Loaded: {train_df.shape}")

# 2. Test 통합 로드 (핵심 수정 사항)
if os.path.exists(TEST_META_PATH):
    test_meta = pd.read_csv(TEST_META_PATH)
    print(f"ℹ️ Reading {len(test_meta)} test files...")
    
    test_dfs = []
    # tqdm으로 진행상황 확인
    for _, row in tqdm(test_meta.iterrows(), total=len(test_meta), desc="Loading Test CSVs"):
        rel_path = row['path']
        
        # 경로 보정: 1) 그대로, 2) 폴더 붙여서, 3) 파일명만으로 찾기
        paths_to_try = [
            rel_path,
            os.path.join(BASE_DIR, rel_path.lstrip("./")),
            os.path.join(BASE_DIR, "test", str(row['game_id']), os.path.basename(rel_path))
        ]
        
        for p in paths_to_try:
            if os.path.exists(p):
                test_dfs.append(pd.read_csv(p))
                break
        
    if test_dfs:
        test_df = pd.concat(test_dfs, ignore_index=True)
        print(f"✅ Test Data Merged: {test_df.shape}")
    else:
        raise FileNotFoundError("Test 파일들을 찾을 수 없습니다.")
else:
    raise FileNotFoundError("test.csv 파일이 없습니다.")

# 3. Match Info 병합
if os.path.exists(MATCH_PATH):
    match_info = pd.read_csv(MATCH_PATH)
    match_subset = match_info[['game_id', 'home_team_id', 'venue']]
    train_df = pd.merge(train_df, match_subset, on='game_id', how='left')
    test_df = pd.merge(test_df, match_subset, on='game_id', how='left')

# 4. 전처리 (결측치 채우기)
def preprocess(df):
    if 'home_team_id' in df.columns:
        df['is_home'] = (df['team_id'] == df['home_team_id']).astype(float)
    else:
        df['is_home'] = 0.5
        
    # V2 로직 작동을 위해 end_x가 없으면 0으로 채움
    if 'end_x' not in df.columns:
        df['end_x'] = 0.0
        df['end_y'] = 0.0
    else:
        df['end_x'] = df['end_x'].fillna(0.0)
        df['end_y'] = df['end_y'].fillna(0.0)
    return df

train_df = preprocess(train_df)
test_df = preprocess(test_df)

ID_COL = 'game_episode' if 'game_episode' in train_df.columns else 'episode_id'
print(f"Data Ready. ID Column: {ID_COL}")

✅ Train Loaded: (356721, 15)
ℹ️ Reading 2414 test files...


Loading Test CSVs: 100%|██████████| 2414/2414 [00:04<00:00, 570.49it/s]


✅ Test Data Merged: (53110, 15)
Data Ready. ID Column: game_episode


In [10]:
# 1. 범주형 변수 정의 (Train 데이터 분석 기반)
TOP_TYPES = ['Pass', 'Carry', 'Recovery', 'Interception', 'Duel', 'Tackle', 
             'Throw-In', 'Clearance', 'Intervention', 'Block', 'Pass_Freekick', 
             'Cross', 'Goal Kick', 'Error', 'Shot']
ALL_RESULTS = ['Successful', 'Unsuccessful', 'On Target', 'Yellow_Card', 
               'Blocked', 'Keeper Rush-Out', 'Low Quality Shot', 'Off Target']

def make_features(group):
    n = len(group)
    
    # --- 1. 기본 좌표 & Context ---
    sx = group['start_x'].values / 105.0
    sy = group['start_y'].values / 68.0
    ex = group['end_x'].values / 105.0
    ey = group['end_y'].values / 68.0
    is_home = group['is_home'].values
    
    # --- 2. 시간 & 속도 (New!) ---
    # time_seconds가 있으면 사용, 없으면 0 처리
    if 'time_seconds' in group.columns:
        times = group['time_seconds'].values
        # 시간 간격 (Delta Time): 현재 - 직전
        # 0번째는 0으로 처리 (혹은 1초 등 기본값)
        dt = np.zeros(n, dtype=np.float32)
        dt[1:] = times[1:] - times[:-1]
        dt = np.maximum(dt, 0.1) # 0으로 나누기 방지 (최소 0.1초)
    else:
        dt = np.ones(n, dtype=np.float32) # 시간 정보 없으면 1초로 가정

    # --- 3. 이동량 & 거리 계산 ---
    dx = ex - sx
    dy = ey - sy
    # 정규화된 거리 말고, 실제 미터 거리 사용 (속도 계산용)
    dist_meter = np.sqrt((dx*105)**2 + (dy*68)**2)
    
    # --- 4. 통계량 (Lagged Features for Leakage Prevention) ---
    # (1) 누적 이동량 (Flow)
    cumsum_dx = np.cumsum(dx) / 105.0
    cumsum_dy = np.cumsum(dy) / 68.0
    
    # (2) 직전 이동 거리 & 누적값 (Lagged)
    # lag_val[i] = val[i-1] (직전 행동의 결과)
    lag_dist_m = np.roll(dist_meter, 1); lag_dist_m[0] = 0
    lag_cumsum_dx = np.roll(cumsum_dx, 1); lag_cumsum_dx[0] = 0
    lag_cumsum_dy = np.roll(cumsum_dy, 1); lag_cumsum_dy[0] = 0
    
    # (3) 직전 속도 (Speed) - New!
    # 직전 거리 / 직전 시간간격 (m/s)
    lag_dt = np.roll(dt, 1); lag_dt[0] = 1.0 # 0 나누기 방지
    lag_speed = lag_dist_m / np.maximum(lag_dt, 0.1)
    
    # --- 5. 선수 정보 (New!) ---
    # 같은 선수가 연속으로 공을 잡았는가? (드리블/키핑 여부)
    if 'player_id' in group.columns:
        p_ids = group['player_id'].values
        # [i]와 [i-1]이 같으면 1, 다르면 0
        is_same = np.zeros(n, dtype=np.float32)
        is_same[1:] = (p_ids[1:] == p_ids[:-1]).astype(np.float32)
    else:
        is_same = np.zeros(n, dtype=np.float32)

    # --- 6. 진행률 & 기타 ---
    progress = np.arange(n) / max(n-1, 1)
    # 후반전 여부 (period_id가 2 이상이면 1)
    is_second_half = (group['period_id'].values > 1).astype(np.float32) if 'period_id' in group.columns else np.zeros(n)
    
    # --- 7. 범주형 변수 One-hot Encoding (New!) ---
    # (1) Type Name (Event 종류)
    # Test 마지막 행에도 'Pass', 'Carry' 등은 주어지므로 사용 가능
    # 미리 정의된 TOP_TYPES에 대해 1/0 매핑
    types_onehot = np.zeros((n, len(TOP_TYPES) + 1), dtype=np.float32) # +1 for Other
    curr_types = group['type_name'].values
    for i, t in enumerate(curr_types):
        if t in TOP_TYPES:
            idx = TOP_TYPES.index(t)
            types_onehot[i, idx] = 1.0
        else:
            types_onehot[i, -1] = 1.0 # Other

    # (2) Result Name (성공 여부)
    # Test 마지막 행이 만약 Pass(Successful)이라면 "성공한 패스의 도착점"을 묻는 것이므로 사용 가능.
    # 만약 비어있다면(nan) 0으로 처리됨.
    results_onehot = np.zeros((n, len(ALL_RESULTS) + 1), dtype=np.float32) # +1 for Nan/Other
    curr_results = group['result_name'].values
    for i, r in enumerate(curr_results):
        if r in ALL_RESULTS:
            idx = ALL_RESULTS.index(r)
            results_onehot[i, idx] = 1.0
        else:
            results_onehot[i, -1] = 1.0 # Nan or Other

    features = []
    
    # --- 8. 시퀀스 결합 ---
    for i in range(n):
        # Scalar Features (Vector Concatenation)
        scalars = [
            sx[i], sy[i],             # 1. 위치
            lag_cumsum_dx[i], lag_cumsum_dy[i], # 2. 누적 흐름 (Safe)
            lag_dist_m[i] / 100.0,    # 3. 직전 거리 (스케일링: 대략 100m로 나눔)
            lag_speed[i] / 10.0,      # 4. 직전 속도 (스케일링: 대략 10m/s로 나눔)
            dt[i] / 10.0,             # 5. 현재 시간 경과 (스케일링)
            progress[i],              # 6. 진행률
            is_home[i],               # 7. 홈 여부
            is_same[i],               # 8. 선수 연속성
            is_second_half[i]         # 9. 후반전 여부
        ]
        
        # Combine: Scalar + Type Onehot + Result Onehot
        feat_vec = np.concatenate([scalars, types_onehot[i], results_onehot[i]])
        
        # (1) Start Node
        features.append(feat_vec)
        
        # (2) End Node (마지막 타겟 제외) -> 여기도 차원 맞춰줘야 함
        if i < n - 1:
            # End Node는 '위치'만 바뀌고 나머지는 그 시점 상태 유지
            # 단, End 시점의 누적/거리는 '현재(i)' 값을 써야 함 (이미 이동했으므로)
            scalars_end = scalars.copy()
            scalars_end[0] = ex[i] # End X
            scalars_end[1] = ey[i] # End Y
            # End Node에는 '이번 이동'의 결과가 반영된 통계 사용 가능
            scalars_end[2] = cumsum_dx[i]
            scalars_end[3] = cumsum_dy[i]
            
            feat_vec_end = np.concatenate([scalars_end, types_onehot[i], results_onehot[i]])
            features.append(feat_vec_end)
            
    return np.array(features, dtype=np.float32)

# 입력 차원 자동 계산
# Scalar(11) + Type(16) + Result(9) = 36
dummy_group = train_df.iloc[:5].copy()
dummy_feat = make_features(dummy_group)
INPUT_DIM = dummy_feat.shape[1]

print(f"✅ Advanced Feature Logic Ready.")
print(f"   Input Dimension: {INPUT_DIM}")
print(f"   (Included: Speed, Time Delta, Player Continuity, Event Type, Result)")

✅ Advanced Feature Logic Ready.
   Input Dimension: 36
   (Included: Speed, Time Delta, Player Continuity, Event Type, Result)


In [11]:
class SoccerDataset(Dataset):
    def __init__(self, df, mode='train'):
        self.mode = mode
        self.episodes = []
        self.targets = []
        self.team_ids = []
        self.episode_ids = []
        
        # 순서 섞임 방지
        grouped = df.groupby(ID_COL, sort=False)
        
        for name, group in tqdm(grouped, desc=f"Dataset ({mode})"):
            if mode == 'train' and len(group) < 2: continue
            
            # Feature Engineering
            if 'time_seconds' in group.columns:
                 seq = make_features(group)
            else:
                 seq = make_v2_features_corrected(group)

            team_id = group.iloc[0]['team_id']
            
            if mode == 'train':
                last = group.iloc[-1]
                self.targets.append([last['end_x']/105.0, last['end_y']/68.0])
                self.episodes.append(seq)
                self.team_ids.append(team_id)
            else:
                self.episodes.append(seq)
                self.team_ids.append(team_id)
                self.episode_ids.append(str(name))

    def __len__(self): return len(self.episodes)
    
    def __getitem__(self, idx):
        seq = torch.FloatTensor(self.episodes[idx])
        if len(seq) > MAX_SEQ_LEN: seq = seq[-MAX_SEQ_LEN:]
        
        if self.mode == 'train':
            # Train 반환: (seq, target, team_id)
            return seq, torch.FloatTensor(self.targets[idx]), self.team_ids[idx]
        else:
            # Test 반환: (seq, team_id, episode_id)
            return seq, self.team_ids[idx], self.episode_ids[idx]

# [핵심 수정] collate_fn 로직 변경
def collate_fn(batch):
    seqs = [b[0] for b in batch]
    lens = torch.LongTensor([len(s) for s in seqs])
    padded = pad_sequence(seqs, batch_first=True, padding_value=0)
    mask = torch.arange(padded.size(1))[None, :] >= lens[:, None]
    
    # 배치 첫 번째 요소로 모드를 판단
    elem = batch[0]
    
    # Test Mode 확인 (3번째 요소가 문자열 ID인 경우)
    if isinstance(elem[2], str):
        # Test: b[1]이 Team ID
        team_ids = torch.LongTensor([b[1] for b in batch])
        episode_ids = [b[2] for b in batch]
        return padded, mask, team_ids, episode_ids
        
    else:
        # Train Mode: b[1]은 Target(좌표), b[2]가 Team ID
        targets = torch.stack([b[1] for b in batch])
        team_ids_train = torch.LongTensor([b[2] for b in batch]) 
        return padded, targets, mask, team_ids_train

# 데이터셋 생성
full_dataset = SoccerDataset(train_df, mode='train')
test_dataset = SoccerDataset(test_df, mode='test')

Dataset (test): 100%|██████████| 2414/2414 [00:01<00:00, 1423.57it/s]


In [ ]:
# ======================================================
# [V5] Sinusoidal Positional Encoding + GELU Activation
# ======================================================
class SinusoidalPositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=100):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        return x + self.pe[:, :x.size(1), :]

class TransformerPredictor(nn.Module):
    def __init__(self, input_dim, d_model, nhead, num_layers, dropout, max_seq_len=MAX_SEQ_LEN):
        super().__init__()
        self.embedding = nn.Linear(input_dim, d_model)

        # [V5] Sinusoidal Positional Encoding (더 안정적인 성능)
        self.pos_encoder = SinusoidalPositionalEncoding(d_model, max_len=max_seq_len)
        self.dropout = nn.Dropout(dropout)

        # [V5] GELU 활성화 함수 적용
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=nhead,
            dim_feedforward=d_model*4,
            dropout=dropout,
            activation='gelu',  # ReLU → GELU
            batch_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        self.fc = nn.Sequential(
            nn.Linear(d_model, d_model // 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(d_model // 2, 2)
        )

    def forward(self, x, mask=None):
        # Embedding
        x = self.embedding(x)

        # Sinusoidal Positional Encoding 추가
        x = self.pos_encoder(x)

        x = self.dropout(x)
        output = self.transformer(x, src_key_padding_mask=mask)

        # Valid Token Mean Pooling
        valid_mask = (~mask).unsqueeze(-1).float()
        mean_out = (output * valid_mask).sum(dim=1) / valid_mask.sum(dim=1).clamp(min=1e-9)
        return self.fc(mean_out)

model = TransformerPredictor(INPUT_DIM, D_MODEL, NHEAD, NUM_LAYERS, DROPOUT).to(DEVICE)
print("✅ [V5] Sinusoidal PE + GELU 적용 완료 (Learnable PE보다 안정적)")

In [ ]:
# ======================================================
# Sinusoidal Positional Encoding 시각화
# ======================================================
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import font_manager, rc

# 한글 폰트 설정
plt.rcParams['font.family'] = 'Malgun Gothic'  # Windows
plt.rcParams['axes.unicode_minus'] = False  # 마이너스 기호 깨짐 방지

def visualize_positional_encoding(max_len=30, d_model=128):
    """
    Sinusoidal Positional Encoding의 패턴을 시각화
    
    각 위치(position)마다 고유한 sin/cos 조합이 생성되어,
    모델이 "3번째 패스"와 "10번째 패스"를 명확히 구분할 수 있습니다.
    """
    # Positional Encoding 생성
    pe = torch.zeros(max_len, d_model)
    position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
    div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
    pe[:, 0::2] = torch.sin(position * div_term)
    pe[:, 1::2] = torch.cos(position * div_term)
    
    pe_array = pe.numpy()
    
    # Figure 생성
    fig = plt.figure(figsize=(18, 10))
    gs = fig.add_gridspec(3, 2, hspace=0.35, wspace=0.3)
    
    # ========================================
    # 1. Positional Encoding 전체 히트맵
    # ========================================
    ax1 = fig.add_subplot(gs[0, :])
    im = ax1.imshow(pe_array.T, cmap='RdBu_r', aspect='auto', vmin=-1, vmax=1)
    ax1.set_xlabel('Position (패스 순서)', fontsize=12, fontweight='bold')
    ax1.set_ylabel('Embedding Dimension', fontsize=12, fontweight='bold')
    ax1.set_title('Sinusoidal Positional Encoding Matrix (30 positions × 128 dimensions)', 
                  fontsize=14, fontweight='bold', pad=15)
    
    # 특정 위치 강조 (3, 10, 20번째)
    for pos in [3, 10, 20]:
        ax1.axvline(x=pos, color='yellow', linewidth=2, linestyle='--', alpha=0.7)
        ax1.text(pos, d_model + 5, f'Pos {pos}', ha='center', fontsize=10, 
                color='yellow', fontweight='bold')
    
    plt.colorbar(im, ax=ax1, label='Encoding Value')
    ax1.grid(False)
    
    # ========================================
    # 2. 각 차원의 주파수 패턴 (선택된 차원들)
    # ========================================
    ax2 = fig.add_subplot(gs[1, 0])
    
    # 다양한 주파수를 보여주기 위해 여러 차원 선택
    dims_to_plot = [0, 2, 8, 16, 32, 64]
    colors_line = plt.cm.viridis(np.linspace(0, 1, len(dims_to_plot)))
    
    for dim, color in zip(dims_to_plot, colors_line):
        ax2.plot(range(max_len), pe_array[:, dim], 
                label=f'Dim {dim}', linewidth=2, color=color, alpha=0.8)
    
    ax2.set_xlabel('Position (패스 순서)', fontsize=11, fontweight='bold')
    ax2.set_ylabel('Encoding Value', fontsize=11, fontweight='bold')
    ax2.set_title('각 차원의 Sinusoidal 패턴 (다양한 주파수)', fontsize=12, fontweight='bold')
    ax2.legend(loc='upper right', fontsize=9)
    ax2.grid(True, alpha=0.3)
    ax2.axhline(y=0, color='black', linewidth=0.8, linestyle='-', alpha=0.3)
    
    # ========================================
    # 3. 특정 위치들의 인코딩 벡터 비교
    # ========================================
    ax3 = fig.add_subplot(gs[1, 1])
    
    positions_compare = [3, 10, 20]
    colors_bar = ['#FF6B6B', '#4ECDC4', '#FFE66D']
    
    x_offset = np.arange(len(positions_compare))
    bar_width = 0.25
    
    # 처음 12개 차원만 보여주기 (가독성)
    dims_show = 12
    for i, pos in enumerate(positions_compare):
        ax3.bar(np.arange(dims_show) + i * bar_width, 
               pe_array[pos, :dims_show], 
               width=bar_width, 
               label=f'Position {pos}',
               color=colors_bar[i],
               alpha=0.8)
    
    ax3.set_xlabel('Embedding Dimension (처음 12개)', fontsize=11, fontweight='bold')
    ax3.set_ylabel('Encoding Value', fontsize=11, fontweight='bold')
    ax3.set_title('다른 위치들의 인코딩 벡터 비교', fontsize=12, fontweight='bold')
    ax3.legend(fontsize=10)
    ax3.grid(True, alpha=0.3, axis='y')
    ax3.set_xticks(np.arange(dims_show) + bar_width)
    ax3.set_xticklabels([f'{i}' for i in range(dims_show)])
    
    # ========================================
    # 4. 위치 간 거리 (유사도) 히트맵
    # ========================================
    ax4 = fig.add_subplot(gs[2, 0])
    
    # 코사인 유사도 계산
    from sklearn.metrics.pairwise import cosine_similarity
    similarity = cosine_similarity(pe_array[:max_len])
    
    im4 = ax4.imshow(similarity, cmap='YlGnBu', aspect='auto')
    ax4.set_xlabel('Position', fontsize=11, fontweight='bold')
    ax4.set_ylabel('Position', fontsize=11, fontweight='bold')
    ax4.set_title('위치 간 코사인 유사도 (각 위치의 고유성)', fontsize=12, fontweight='bold')
    plt.colorbar(im4, ax=ax4, label='Cosine Similarity')
    
    # 대각선 (자기 자신)은 항상 1이고, 멀어질수록 유사도가 낮아짐을 보여줌
    ax4.text(max_len//2, max_len - 2, 
            '대각선: 자기 자신 (유사도=1)\n멀수록 낮은 유사도',
            ha='center', va='top', 
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.8),
            fontsize=9)
    
    # ========================================
    # 5. 주파수 분석 (FFT)
    # ========================================
    ax5 = fig.add_subplot(gs[2, 1])
    
    # 몇 개 차원의 FFT 계산
    dims_fft = [0, 4, 16, 64]
    
    for dim in dims_fft:
        signal = pe_array[:, dim]
        fft = np.fft.fft(signal)
        freqs = np.fft.fftfreq(len(signal))
        
        # 양수 주파수만
        pos_mask = freqs > 0
        ax5.plot(freqs[pos_mask], np.abs(fft[pos_mask]), 
                label=f'Dim {dim}', linewidth=2, alpha=0.7)
    
    ax5.set_xlabel('주파수 (Frequency)', fontsize=11, fontweight='bold')
    ax5.set_ylabel('강도 (Magnitude)', fontsize=11, fontweight='bold')
    ax5.set_title('주파수 도메인 분석 (각 차원의 주파수 성분)', fontsize=12, fontweight='bold')
    ax5.legend(fontsize=9)
    ax5.grid(True, alpha=0.3)
    ax5.set_xlim([0, 0.5])
    
    # 전체 타이틀
    fig.suptitle('🎯 Sinusoidal Positional Encoding 시각화\n"각 위치(패스)마다 고유한 sin/cos 지문을 가집니다"', 
                 fontsize=16, fontweight='bold', y=0.995)
    
    plt.savefig('positional_encoding_visualization.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    # ========================================
    # 요약 통계 출력
    # ========================================
    print("\n" + "="*60)
    print("📊 Positional Encoding 요약 통계")
    print("="*60)
    print(f"Max Length (최대 시퀀스 길이): {max_len}")
    print(f"D_Model (임베딩 차원): {d_model}")
    print(f"Encoding 범위: [{pe_array.min():.4f}, {pe_array.max():.4f}]")
    print(f"\n위치 간 평균 코사인 유사도:")
    for i, pos1 in enumerate([0, 5, 10, 15, 20, 25]):
        if pos1 >= max_len: continue
        avg_sim = np.mean([similarity[pos1, pos2] for pos2 in range(max_len) if pos2 != pos1])
        print(f"  Position {pos1:2d}: {avg_sim:.4f} (다른 위치들과의 평균 유사도)")
    
    print(f"\n✅ 각 위치는 고유한 sin/cos 패턴을 가지므로")
    print(f"   트랜스포머가 '3번째 패스'와 '10번째 패스'를 명확히 구분할 수 있습니다!")
    print("="*60 + "\n")

# 실행
visualize_positional_encoding(max_len=MAX_SEQ_LEN, d_model=D_MODEL)

In [ ]:
# ======================================================
# [V5 GELU] 고급 학습 전략 적용
# - Cosine Annealing Scheduler
# - Warmup Learning Rate
# - Gradient Clipping
# - Huber Loss
# ======================================================
from torch.optim.lr_scheduler import CosineAnnealingLR

SEEDS = [42, 2024, 777]
print(f">>> [V5 GELU] Advanced Training Start (3 Models)")

# 히스토리 저장용
all_histories = []

for i, seed in enumerate(SEEDS):
    print(f"\n🚀 [Model {i+1}/3] Training with Seed {seed}...")
    seed_everything(seed)
    
    # 데이터셋 분할
    train_idx, val_idx = train_test_split(range(len(full_dataset)), test_size=0.2, random_state=seed)
    train_loader = DataLoader(torch.utils.data.Subset(full_dataset, train_idx), batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
    val_loader = DataLoader(torch.utils.data.Subset(full_dataset, val_idx), batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)

    # 모델 초기화
    model = TransformerPredictor(INPUT_DIM, D_MODEL, NHEAD, NUM_LAYERS, DROPOUT).to(DEVICE)
    optimizer = optim.AdamW(model.parameters(), lr=LR_BASE, weight_decay=1e-4)
    
    # Cosine Annealing Scheduler (Warmup 이후 적용)
    scheduler = CosineAnnealingLR(optimizer, T_max=EPOCHS_BASE - WARMUP_EPOCHS, eta_min=1e-6)
    
    # Huber Loss (MSE보다 이상치에 강함)
    criterion = nn.HuberLoss(delta=1.0)
    
    best_dist = float('inf')
    
    # 히스토리 저장
    history = {
        'train_loss': [],
        'val_dist': [],
        'lr': []
    }
    
    for epoch in range(EPOCHS_BASE):
        # === Warmup Learning Rate ===
        if epoch < WARMUP_EPOCHS:
            lr = LR_BASE * (epoch + 1) / WARMUP_EPOCHS
            for param_group in optimizer.param_groups:
                param_group['lr'] = lr
        
        # === Training ===
        model.train()
        train_loss = 0.0
        for seqs, targets, mask, _ in train_loader:
            seqs, targets, mask = seqs.to(DEVICE), targets.to(DEVICE), mask.to(DEVICE)
            
            optimizer.zero_grad()
            pred = model(seqs, mask)
            loss = criterion(pred, targets)
            loss.backward()
            
            # Gradient Clipping
            torch.nn.utils.clip_grad_norm_(model.parameters(), GRAD_CLIP)
            
            optimizer.step()
            train_loss += loss.item()
        
        # === Validation ===
        model.eval()
        dists = []
        with torch.no_grad():
            for seqs, targets, mask, _ in val_loader:
                seqs, targets, mask = seqs.to(DEVICE), targets.to(DEVICE), mask.to(DEVICE)
                pred = model(seqs, mask)
                p_real = pred.cpu().numpy() * np.array([105.0, 68.0])
                t_real = targets.cpu().numpy() * np.array([105.0, 68.0])
                dists.extend(np.sqrt(np.sum((p_real - t_real)**2, axis=1)))
        
        avg_dist = np.mean(dists)
        avg_train_loss = train_loss / len(train_loader)
        
        # Scheduler Step (Warmup 이후)
        if epoch >= WARMUP_EPOCHS:
            scheduler.step()
        
        current_lr = optimizer.param_groups[0]['lr']
        
        # 히스토리 저장
        history['train_loss'].append(avg_train_loss)
        history['val_dist'].append(avg_dist)
        history['lr'].append(current_lr)
        
        # 로그 출력
        if (epoch + 1) % 5 == 0 or epoch < 5:
            print(f"  [Epoch {epoch+1:2d}/{EPOCHS_BASE}] "
                  f"Loss: {avg_train_loss:.4f} | "
                  f"Val Dist: {avg_dist:.4f}m | "
                  f"LR: {current_lr:.6f}")
        
        # Best Model 저장
        if avg_dist < best_dist:
            best_dist = avg_dist
            torch.save(model.state_dict(), f'base_model_{i}.pth')
    
    history['best_dist'] = best_dist
    history['seed'] = seed
    all_histories.append(history)
    
    print(f"  ✅ Model {i} Finished. Best Val Dist: {best_dist:.4f}m")

print("\n✅ All 3 Models Trained Successfully!")

In [ ]:
# ======================================================
# [V5 Sinusoidal PE + GELU] ID 매핑을 통한 안전한 제출 (Merge 방식)
# ======================================================
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)

# 모델 로드 (3개 앙상블)
models = []
for i in range(3):
    m = TransformerPredictor(INPUT_DIM, D_MODEL, NHEAD, NUM_LAYERS, DROPOUT).to(DEVICE)
    m.load_state_dict(torch.load(f'base_model_{i}.pth'))
    m.eval()
    models.append(m)

print(f">>> [V5 Sinusoidal PE + GELU] Weighted Ensemble Inference...")
results = [] # [ID, Pred_X, Pred_Y] 저장할 리스트
weights = [0.25, 0.25, 0.5]

with torch.no_grad():
    # 로더에서 episode_ids를 같이 받음
    for seqs, mask, team_ids, episode_ids in tqdm(test_loader):
        seqs, mask = seqs.to(DEVICE), mask.to(DEVICE)
        
        m1 = models[0](seqs, mask).cpu().numpy()
        m2 = models[1](seqs, mask).cpu().numpy()
        m3 = models[2](seqs, mask).cpu().numpy()
        
        avg_pred = (m1 * weights[0]) + (m2 * weights[1]) + (m3 * weights[2])
        
        # 결과 저장 (ID와 예측값을 짝지음)
        for i, eid in enumerate(episode_ids):
            px = avg_pred[i, 0] * 105.0
            py = avg_pred[i, 1] * 68.0
            results.append({'game_episode': eid, 'pred_x': px, 'pred_y': py})

# 1. 예측 결과 DataFrame 생성
pred_df = pd.DataFrame(results)

# 2. 제출 양식 로드
SUBMISSION_PATH = "open_track1/sample_submission.csv"
if os.path.exists(SUBMISSION_PATH):
    sub = pd.read_csv(SUBMISSION_PATH)
else:
    sub = pd.read_csv(TEST_META_PATH)
    # 컬럼명 통일
    col_map = {'episode_id': 'game_episode'}
    sub = sub.rename(columns=col_map)
    sub = sub[['game_episode']]

print(f"Submission Shape: {sub.shape}, Prediction Shape: {pred_df.shape}")

# 3. [핵심] ID 기준으로 합치기 (Left Join)
# 제출 파일의 순서를 유지하면서, 예측값을 옆에 붙임
final_sub = pd.merge(sub[['game_episode']], pred_df, on='game_episode', how='left')

# 4. 컬럼명 변경 및 결측치 확인
final_sub = final_sub.rename(columns={'pred_x': 'end_x', 'pred_y': 'end_y'})

# 혹시 매칭 안 된 게 있는지 확인
nan_count = final_sub.isnull().sum().sum()
if nan_count > 0:
    print(f"⚠️ 경고: {nan_count}개의 결측치가 발생했습니다. (ID 불일치)")
    final_sub = final_sub.fillna(50.0) # 중앙값으로 땜빵

# 5. 저장
filename = "v5_sinusoidal_gelu_submit.csv"
final_sub.to_csv(filename, index=False)
print(f"✅ Submission Saved: {filename}")
print(f"✅ [V5] Sinusoidal PE + GELU 적용 완료")
print(final_sub.head())

In [ ]:
# ======================================================
# 학습 곡선 시각화
# ======================================================
import matplotlib.pyplot as plt

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('[V5 GELU] Training History', fontsize=16, fontweight='bold')

# 모델별 색상
colors = ['#1f77b4', '#ff7f0e', '#2ca02c']
labels = [f'Model {i+1} (Seed {h["seed"]})' for i, h in enumerate(all_histories)]

# 1. Training Loss
ax1 = axes[0, 0]
for i, history in enumerate(all_histories):
    epochs = range(1, len(history['train_loss']) + 1)
    ax1.plot(epochs, history['train_loss'], label=labels[i], color=colors[i], linewidth=2)
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Huber Loss')
ax1.set_title('Training Loss')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 2. Validation Distance
ax2 = axes[0, 1]
for i, history in enumerate(all_histories):
    epochs = range(1, len(history['val_dist']) + 1)
    ax2.plot(epochs, history['val_dist'], label=labels[i], color=colors[i], linewidth=2)
    # Best 지점 표시
    best_idx = np.argmin(history['val_dist'])
    best_val = history['val_dist'][best_idx]
    ax2.scatter(best_idx + 1, best_val, color=colors[i], s=100, zorder=5, marker='*')
    ax2.annotate(f'{best_val:.2f}m', 
                 xy=(best_idx + 1, best_val), 
                 xytext=(5, 5), 
                 textcoords='offset points',
                 fontsize=9,
                 color=colors[i])
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Distance (m)')
ax2.set_title('Validation Distance')
ax2.legend()
ax2.grid(True, alpha=0.3)

# 3. Learning Rate Schedule
ax3 = axes[1, 0]
history = all_histories[0]  # LR은 모두 동일하므로 첫 번째 것만
epochs = range(1, len(history['lr']) + 1)
ax3.plot(epochs, history['lr'], color='purple', linewidth=2)
ax3.axvline(x=WARMUP_EPOCHS, color='red', linestyle='--', label=f'Warmup End (Epoch {WARMUP_EPOCHS})')
ax3.set_xlabel('Epoch')
ax3.set_ylabel('Learning Rate')
ax3.set_title('Learning Rate Schedule (Warmup + Cosine Annealing)')
ax3.legend()
ax3.grid(True, alpha=0.3)
ax3.set_yscale('log')

# 4. Best Distances 비교
ax4 = axes[1, 1]
best_dists = [h['best_dist'] for h in all_histories]
seeds = [h['seed'] for h in all_histories]
bars = ax4.bar(range(len(best_dists)), best_dists, color=colors)
ax4.set_xticks(range(len(best_dists)))
ax4.set_xticklabels([f'Model {i+1}\n(Seed {s})' for i, s in enumerate(seeds)])
ax4.set_ylabel('Best Validation Distance (m)')
ax4.set_title('Best Performance per Model')
ax4.grid(True, alpha=0.3, axis='y')

# 각 바 위에 값 표시
for i, (bar, dist) in enumerate(zip(bars, best_dists)):
    height = bar.get_height()
    ax4.text(bar.get_x() + bar.get_width()/2., height,
             f'{dist:.4f}m',
             ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.savefig('v5_gelu_training_history.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\n📊 학습 곡선 저장 완료: v5_gelu_training_history.png")
print(f"\n📈 최종 결과:")
for i, h in enumerate(all_histories):
    print(f"  Model {i+1} (Seed {h['seed']:4d}): {h['best_dist']:.4f}m")
print(f"  평균: {np.mean([h['best_dist'] for h in all_histories]):.4f}m")